In [1]:
#This is the learning algoritm which would be used. It has skipping, multi-threading enabled. 

In [35]:
import numpy as np
import pandas as pd
from scipy.stats import bernoulli
from sklearn import linear_model
import threading

# Contexts

In [36]:
import pickle
'''
Context data for learning
'''
class Context:
    """
    
    """
    # Student Preferences: 
    # Visual (S_V) , Text (S_T) , Demo-based (S_D) , Practical (S_P), Step-by-step (S_S) ,Activity / Task based (S_AT), 
    # Lecture (S_L) , Audio (S_A) , Self-evaluation (S_SE) , Pre-assessment (S_PA)
    # Students preference to learning via various ways can also be evaluated on a scale of 10, rather being binary. 
    def getStudentContext(self):
        """
        
        """
        return self.studentContext
    
    def setStudentContext(self):
        """
        
        """
        with open('student.pickle', 'rb') as student_file:
            self.studentContext= pickle.load(student_file)
    
    # Content Features 
    # Ease of understanding (C_E) , Simple / Intuitive (C_I) , Surface / In-depth (C_ID) , Brief / Concise (C_C), 
    # Thorough (C_T), Preference / Well reviewed / Well rated (C_R) , Theoritical / Abstract (C_A), 
    # Practical / Hands on (C_P), Experimental / Task-based (C_ETB)
    # Content preference to learning via various ways can also be evaluated on a scale of 10, rather being binary. 
    def getContentContext(self):
        """
        
        """
        return self.contentContext
   
    def setContentContext(self):
        """
        
        """
        with open('content.pickle', 'rb') as content_file:
            self.contentContext= pickle.load(content_file)
        
    def getTopic(self):
        """
        
        """
        return self.topic
    
    def setTopic(self):
        """
        
        """
        with open('topic.pickle', 'rb') as topic_file:
            self.topic = pickle.load(topic_file)
    
    def getTopicContent(self):
        """
         Gets the topic content. topic_content is a map of topics to content. So for every topic, it gives the content 
         available for the topic. In education parlance, for any given topics, it shows the different ways of teaching this
         topic (via contents)
        """
        return self.topic_content
    
    def setTopicContent(self):
        """
        Sets the topic_content variable to the one in the serialized object. topic_content is a map of topics to content. So
        for every topic, it gives the content available for the topic. In education parlance, for any given topics, it shows
        the different ways of teaching this topic (via contents)
        """
        with open('topic_content.pickle', 'rb') as topic_content_file:
            self.topic_content= pickle.load(topic_content_file)
                
    def prepareContext(self,studentContext,contentContext):
        """
           Given the student & content context available for a round, this method combines them to form a single contextual
           variable
        """
        context = pd.DataFrame() 
        for content in list(contentContext.index):
            c = pd.Series()
            c = c.append([studentContext,contentContext.loc[content]]) # Combine student & content. 
            c['Content_id'] = content
            context = context.append(c, ignore_index=True)
        context = context.set_index('Content_id')
        return context
    
    
    def loadData(self):
        """
        This is a dummy method used to test data generation. Its not invoked in the main program.
        """
        self.setStudentContext()
        self.setContentContext()
        self.setTopic()
        self.setTopicContent()
        print(self.getStudentContext())
        print(type(self.getStudentContext()))
        print('*********************************')
        print(self.getContentContext())
        print(type(self.getContentContext()))
        print('*********************************')
        print(self.getTopic())
        print(type(self.getTopic()))
        print('*********************************')
        print(self.getTopicContent())
        print(type(self.getTopicContent()))
                
c_test = Context()
c_test.loadData()

    S_V   S_T   S_D   S_P   S_S  S_AT   S_L   S_A  S_SE  S_PA
0  0.57  0.90  0.41  0.21  0.71  0.33  0.03  0.68  0.42  0.22
1  0.47  0.02  0.65  0.23  0.68  0.67  0.80  0.40  0.10  0.85
2  0.84  0.93  0.30  0.93  0.91  0.58  0.01  0.27  0.43  0.89
3  0.80  0.98  0.66  0.02  0.10  0.15  0.67  0.18  0.22  0.75
4  0.83  0.22  0.28  0.51  0.68  0.06  0.73  0.55  0.90  0.08
<class 'pandas.core.frame.DataFrame'>
*********************************
        C_E   C_I  C_ID   C_C   C_T   C_R   C_A   C_P  C_ETB
C_1_1  0.27  0.59  0.56  0.42  0.82  0.27  0.84  0.22   0.75
C_1_2  0.14  0.41  0.80  0.91  0.41  0.84  0.09  0.70   0.73
C_1_3  0.48  0.77  0.98  0.86  0.75  0.90  0.79  0.11   0.32
C_1_4  0.48  0.64  0.35  0.63  0.45  0.10  0.09  0.83   0.11
C_2_1  0.97  0.92  0.99  0.22  0.90  0.34  0.43  0.50   0.27
C_2_2  0.52  0.00  0.40  0.97  0.21  0.80  0.73  0.41   0.11
C_2_3  0.58  0.83  0.01  0.74  0.51  0.31  0.98  0.83   0.99
C_3_1  0.76  0.02  0.53  0.23  0.62  0.11  0.50  0.02   0.04
C_3_2  

# Skip Classifier

In [37]:
# Online Stochastic Gradient Descent. This classifier decides whether or not to skip to the next topic. 
# TO-DO : Change loss functions (Log,Hinge,Others) to find if they impact performance. Try different values of parameters 
# For instance SGD has a parameter alpha, SVM has a parameter C. To optimize, you can train a mini-batch of samples, 
# rather than one data point at a time. Try different values of learning_rate . Look at the class_weight parameter if you 
# want to give more weight to samples of one class over the other. Need to understand about warm_start parameter
# We need to record predictions made by the classifier to evaluate its performance over rounds 

class SkipClassifier:
    """
    
    """
    def __init__(self):
        self.clf = linear_model.SGDClassifier()
        
    def check_fitted(self,clf): 
        """
        
        """
        return hasattr(clf, "classes_")
    
    def train(self,student,pta,next_topic_pta,label):
        """
        
        """
        X = pd.Series()
        X = X.append([student,pd.Series([pta,next_topic_pta],index=['pta','next_topic_pta'])])
        X = np.array([X.values])
        Y = np.array([label])
        clf = self.clf.partial_fit(X,Y,classes=np.array([0,1]))
            
    def predict(self,student,pta,next_topic_pta):
        """
        
        """
        X = pd.Series()
        X = X.append([student,pd.Series([pta,next_topic_pta],index=['pta','next_topic_pta'])])
        if self.check_fitted(self.clf):
            Y = self.clf.predict([X.values])[0]
            confidence_score = self.clf.decision_function([X.values])[0]
        else:
            Y = 0
            confidence_score = 0
        #print('Skip Prediction : {0} & Confidence Score : {1}'.format(Y,confidence_score))
        # Get confidence score 
        return Y , confidence_score

# Skip Topic

In [39]:
class SkipTopic:
    
    def __init__(self):
        """
        
        """
        self.skipClassifier = SkipClassifier()
        self.confidence_threshold = 60 # It the confidence score returned by the classifier is greater than this, then we trust in the decision made by the classifier. 
        self.skipTopic_lock = threading.Lock()
                
    def skipTopic(self,student,pta,topic_number,context_obj,topic_content,linUCB):
        """
        
        """
        contentContext = context_obj.getContentContext() # Get the content dataframe.
#         print("contentContext : " , contentContext )
        topic = context_obj.getTopic() # Get the topic list. 
        current_topic_index = topic.index(topic_number) # Get the index number of the current topic
        next_topic_index = current_topic_index + 1
        next_topic = '' # Initialized to make it accessible outside the if statement. 
        if next_topic_index < len(topic): # Check to see if we're going out of bounds
            next_topic = topic[next_topic_index]
            next_topic_contents = topic_content[next_topic]
            t_c = contentContext.loc[next_topic_contents]
            X = context_obj.prepareContext(student,t_c)
            arm_pulled , next_topic_pta = linUCB.expectedPayoff(X,next_topic_contents)
        else:
            # Will be going out of bounds. Current topic is the last topic. No more topics to complete. 
            next_topic_pta = 0
            #return # There is no topic to skip to. OR is might be better to exit the system. 
        skip_decision , confidence_score = self.skipClassifier.predict(student,pta,next_topic_pta)
        if skip_decision and confidence_score < self.confidence_threshold:
#             with self.skipTopic_lock:
#                 print('Actual decision made by classifier : ' , skip_decision)
#                 print('Confidence score returned is {0}, which is less than threshold {1}'.format(confidence_score , self.confidence_threshold))
            skip_decision = 0
        return skip_decision,next_topic_pta
        
    def setLabel(self,skip_decision,actual_payoff):
        """
        
        """
        if skip_decision == 0 and actual_payoff == 0:
            label = 1
        if skip_decision == 0 and actual_payoff == 1:
            label = 0
        return label
    
    def train(self,student,pta,pta_next_topic,label):
        """
        
        """
        self.skipClassifier.train(student,pta,pta_next_topic,label)

# Nature / Environment / Universe

In [40]:
class Oracle : 
    """
    
    """
    def setParameters(self, contexts , arms):
        """
        
        """
        parameters = np.random.uniform(size=(len(arms) , len(contexts)))
        # Normalize parameters
        for i in range(parameters.shape[0]): # Have it in a list comprehension.
            parameters[i] = parameters[i] / np.sum(parameters[i])
        self.theta_df = pd.DataFrame(data = parameters ,  index = arms , columns = contexts , dtype= np.float)

# Learning Algorithm

In [42]:
class LinUCB:
    """
    
    """
    def __init__(self,alpha=0.5):
        """
        
        """
        self.alpha = alpha # Hyper parameter required for LinUCB to adjust confidence bounds.
        self.arm_params = {} # Maps content to arm object
        self.linUCB_lock = threading.Lock()
                
    def expectedPayoff(self,contexts,arms):
        """
        
        """
        arms_payoff = list()
        for arm in arms:
            X = contexts.loc[arm] # Give student & content context for an arm 
            if arm not in self.arm_params: # If new content is added, then parameters would be created for it. 
                self.arm_params[arm] = Arm(len(X.index)) # Arm class below, has arm specific parameters
            arm_obj = self.arm_params[arm]
            theta = self.getTheta(arm_obj) # Arm parameter. 
            pta = self.getPta(X , arm_obj) # pta : pay-off/reward at round 't' for arm 'a'. 
            arms_payoff.append(pta)
        expected_payoff = np.max(arms_payoff) # To be used a input data for skip algorithm
        arm_index = np.argmax(arms_payoff) # Find the index of the arm which max pay-off
        arm_pulled = arms[arm_index] # Give me the arm with max pay-off
#         with self.linUCB_lock:
#             print('arms_payoff : ', arms_payoff) # Expected pay-off of all arms. 
#             print('Index of arm with max payoff : ', arm_index)
#             print('Arm pulled : ', arm_pulled)    
        return arm_pulled,expected_payoff    
        
    def getTheta(self,arm): # Theta is used to compute the mean reward for an arm 
        """
        
        """
        arm.theta = np.dot(arm.Ainv , arm.b) # A vector
        return arm.theta
    
    def getMean(self, context , arm):
        """
        
        """
        mean = np.dot(arm.theta.T , context)
#         print('mean : {0} type : {1}'.format(mean,type(mean)))
        return mean
        
    def getUCB(self , context ,arm):
        """
        
        """
        ucb = np.sqrt(np.dot(np.dot(context.T , arm.Ainv) , context))
        return ucb
    
    def getPta(self, context , arm):
        """
        
        """
        payoff = self.getMean(context,arm) + self.alpha * self.getUCB(context , arm)
        return payoff
    
    def updateParams(self, arm , context, reward):
        """
        
        """
        arm_obj = self.arm_params[arm]
        arm_obj.A += np.outer(context,context.T)
        arm_obj.b += reward * context
        arm_obj.Ainv = np.linalg.inv(arm_obj.A)               
    
class Arm:
    """
    
    """
    def __init__(self,dimensions):
        """
        
        """
        self.A = np.identity(dimensions)
        self.b = np.zeros(dimensions)
        self.Ainv = np.linalg.inv(self.A)
        self.theta = np.dot(self.Ainv , self.b)

# Simulator

In [54]:
class Simulator:
    """
    It represents the teaching system. Several students log into it to take courses. 
    """
    
    def __init__(self):
        """
        Initialize the teaching system by loading data about students, topics & content. Also, initialize other objects to
        be used with the learning system, such as the learning algorithm (LinUCB), the omniscient policy (Oracle) , skip topic
        to optimize rewards, by skipping topics which have content with low expected rewards. 
        """
        self.context = Context()
        self.context.setStudentContext()
        self.context.setContentContext()
        self.context.setTopic()
        self.context.setTopicContent()
        self.oracle = Oracle()
        self.linUCB = LinUCB()
        self.skipTopic = SkipTopic()
        self.simulator_lock = threading.Lock()
        self.rounds=0
        self.logs = pd.DataFrame(columns = ['student_number','topic','arm_pulled','pay-off','pay-off_next_topic','skip_decision'
                                            ,'reward']) 

    def getPayoff(self,X,arm,pta):
        """
        Input : 
        
        X : Context vector for the round. 
        arm : Arm to be pulled / Content to be shown
        pta : Payoff at round 't' for pulling arm 'a'
        
        Returns : 
        
        reward : Reward / Feedback from student for the content shown / arm pulled
        """
        arm_theta = self.oracle.theta_df.loc[arm] #Get parameters for the arm predicted by the learning algo
        expected_reward = pd.Series.dot(X,arm_theta) # Vector dim : (1 * d) (d * 1).
        reward = bernoulli.rvs(size=1,p=expected_reward)[0] # Simulate student's response
#         with self.simulator_lock:
#             print('Actual Reward : ', reward)        
        return reward
    
    def takeCourse(self,student_number,studentContext,contentContext,topic,topic_content):
        """
        Inputs : 
        
        student_number : Student Id 
        studentContext : Student context vector. 
        contentContext : Contents context. This has context of all contents for the topic. 
        topic : All the topics to be taught as part of the course. 
        topic_content : Relates all topics to the contents available for every topic        
        """
        for i in topic:
            contents = topic_content[i] # You now have all arm associated with the topic 't'
            t_c = contentContext.loc[contents]
#             with self.simulator_lock:
#                 print('Student Number {0} is on topic {1}'.format(student_number,i))
            #print('takeCourse : Student Number taking course : ', student_number)
            contexts = self.context.prepareContext(studentContext,t_c)
            arms = list(t_c.index)
            while arms:
                arm , pta = self.linUCB.expectedPayoff(contexts,arms)
                skip_decision , pta_next_topic = self.skipTopic.skipTopic(studentContext,pta,i,self.context,topic_content,self.linUCB)
                if skip_decision:
                    break # Decision is to skip. Hence, we won't pull the arm. 
                else:
                    actual_payoff = self.getPayoff(contexts.loc[arm],arm,pta)
                    log = pd.Series([student_number,i,arm,pta,pta_next_topic,skip_decision,actual_payoff], 
                                        index=['student_number','topic','arm_pulled','pay-off',
                                                'pay-off_next_topic','skip_decision','reward'])
                    with self.simulator_lock:
                        self.rounds+=1
                        print('Student {0} is on topic {1} is shown content {2} feedback recd is {3}. Expected Pay-off of this arm is {4}, compared to expected pay-off of next round is {5}. Decision of skip classifier is {6}'
                              .format(student_number,i,arm,actual_payoff,pta,pta_next_topic,skip_decision))
                        self.logs = self.logs.append(log , ignore_index=True)
                    self.linUCB.updateParams(arm,contexts.loc[arm], actual_payoff)
                    label = self.skipTopic.setLabel(skip_decision,actual_payoff) 
                    self.skipTopic.train(studentContext,pta,pta_next_topic,label)
                if actual_payoff != 1:
                    arms.remove(arm)
                else:
                    break # Move to the next topic 

    def main(self):
        """
        
        """
        studentContext = self.context.getStudentContext() # Student dataframe
        contentContext = self.context.getContentContext() # Content Dataframe
        topic = self.context.getTopic() # List of topics. 
        topic_content = self.context.getTopicContent() # Topics Data, which includes topics to content mapping.
        features = list(studentContext.columns) + list(contentContext.columns)
        self.oracle.setParameters(features , contentContext.index)
        student_thread = list()
        for student_number , student in studentContext.iterrows():
            t = threading.Thread(target=self.takeCourse, args=(student_number,student,contentContext,topic,topic_content))
            student_thread.append(t)
            t.daemon = True # classifying as a daemon, so they will die when the main dies. 
            # Not sure whether we should have daemon. In which situation would it be good to have a daemon thread. What is 
            # the main thread finishes & one of the student is still working through the course. Based on my current 
            # understanding, in that case, the student would not be able to complete the course, since the main thread 
            # is completed. Need to check our understanding
            t.start() # begins, must come after daemon definition
        for t in student_thread: # This is done to ensure, we proceed to save the logs only after all students have completed the course. 
            t.join()
        self.logs.to_csv('logs',index=False)
        print('Total Number of rounds : ', self.rounds)  

simulator = Simulator()
simulator.main()

Student 0 is on topic T_1 is shown content C_1_3 feedback recd is 1. Expected Pay-off of this arm is 1.3493146408454924, compared to expected pay-off of next round is 1.3422741895752894. Decision of skip classifier is 0
Student 3 is on topic T_1 is shown content C_1_3 feedback recd is 1. Expected Pay-off of this arm is 1.3960211316452198, compared to expected pay-off of next round is 1.3892174055920836. Decision of skip classifier is 0
Student 2 is on topic T_1 is shown content C_1_3 feedback recd is 1. Expected Pay-off of this arm is 1.5308739334118926, compared to expected pay-off of next round is 1.5246720958947206. Decision of skip classifier is 0
Student 1 is on topic T_1 is shown content C_1_3 feedback recd is 0. Expected Pay-off of this arm is 1.394175383515288, compared to expected pay-off of next round is 1.3873626058100312. Decision of skip classifier is 0
Student 4 is on topic T_1 is shown content C_1_3 feedback recd is 1. Expected Pay-off of this arm is 1.5660390818957195, 

In [55]:
df = pd.read_csv('logs')
df

,student_number,topic,arm_pulled,pay-off,pay-off_next_topic,skip_decision,reward
0,0,T_1,C_1_3,1.349315,1.342274,0,1
1,3,T_1,C_1_3,1.396021,1.389217,0,1
2,2,T_1,C_1_3,1.530874,1.524672,0,1
3,1,T_1,C_1_3,1.394175,1.387363,0,0
4,4,T_1,C_1_3,1.566039,1.392139,0,1
5,0,T_2,C_2_3,1.342274,1.025731,0,1
6,1,T_1,C_1_2,1.294314,1.387363,0,0
7,3,T_2,C_2_3,1.389217,1.086439,0,0
8,2,T_2,C_2_3,1.524672,1.255010,0,0
9,4,T_2,C_2_1,1.364075,1.090172,0,0
